In [ ]:
!pip install google-cloud-bigquery

In [ ]:
import os
os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = "/content/dbt-project.json"

#Część 1

##Zadanie 1:

W jaki sposób połączył(a/e)ś ze sobą dane. Możesz przedstawić relacje między tabelami, pokazać model danych lub opowiedzieć nam w dowolny sposób jakie było Twoje podejście do połączenia danych z różnych tabel.


1.	Dane załadowałem do BigQuery jako cztery osobne tabele w dedykowanym datasecie.
2.	Ustaliłem klucze łączące tabele:
    - orders_final.id = payment_transactions_final.order_id
    - orders_final.product_id = customer_package_deliveries_final.id
    - customer_package_deliveries_final.id = customer_package_delivery_products_final.customer_package_delivery_id
3.	Tabelę orders_final potraktowałem jako centralną i do niej dołączałem pozostałe dane.
4.	Do łączenia użyłem LEFT JOIN, aby zachować pełną historię zamówień, nawet bez płatności lub dostawy.
5.	Dzięki temu stworzyłem spójny model danych obejmujący cały proces: od zamówienia, przez płatność, po dostawę i produkty.
6.	Model ma relacje 1:N, co pozwala analizować wiele płatności i produktów w ramach jednego zamówienia.
7.	Cały proces jest powtarzalny, czytelny i łatwy do odtworzenia w BigQuery.
8.	Takie rozwiązanie wybrałem, ponieważ zapewnia pełny obraz procesów biznesowych i umożliwia elastyczne analizy w jednym, połączonym modelu danych.

### Relacje tabel




In [ ]:
# orders_final
#    ├── id ----------------------> payment_transactions_final.order_id
#    │
#    └── product_id --------------> customer_package_deliveries_final.id
#                                      │
#                                      └── id ------------------> customer_package_delivery_products_final.customer_package_delivery_id

###Zapytanie do BQ

In [ ]:
from google.cloud import bigquery
client = bigquery.Client()

query = """
SELECT
  -- orders_final.csv – informacje o złożonych zamówieniach
  o.id AS order_id,
  o.number AS order_number,
  o.customer_id,
  o.product_id,
  o.price AS order_price,
  o.product_type,
  o.buy_method AS order_buy_method,
  o.discount_code_id,
  o.created_at AS order_created_at,
  o.updated_at AS order_updated_at,
  o.deleted_at AS order_deleted_at,
  o.created_by AS order_created_by,

  -- payment_transactions_final.csv – informacje o płatnościach za zamówienia
  pt.id AS payment_id,
  pt.order_id AS payment_order_id,
  pt.provider,
  pt.status AS payment_status,
  pt.customer_status,
  pt.customer_stage,
  pt.buy_method AS payment_buy_method,
  pt.price AS payment_price,
  pt.created_at AS payment_created_at,
  pt.updated_at AS payment_updated_at,
  pt.deleted_at AS payment_deleted_at,

  -- customer_package_deliveries_final.csv – informacje o wysyłkach Paczek
  d.id AS delivery_id,
  d.number AS delivery_number,
  d.customer_package_id,
  d.status AS delivery_status,
  d.date AS delivery_date,
  d.buy_method AS delivery_buy_method,
  d.delivery_courier_type,
  d.delivery_customer_first_name,
  d.created_at AS delivery_created_at,
  d.updated_at AS delivery_updated_at,
  d.deleted_at AS delivery_deleted_at,

  -- customer_package_delivery_products_final.csv – szczegóły dotyczące produktów w Paczkach.
  p.id AS product_row_id,
  p.customer_package_delivery_id,
  p.product_id AS delivery_product_id,
  p.type AS product_type,
  p.quantity,
  p.pack_quantity,
  p.migration_id,
  p.wms_id,
  p.created_at AS product_created_at,
  p.updated_at AS product_updated_at,
  p.deleted_at AS product_deleted_at

FROM `coral-shift-440513-f3.meat_package.orders_final` o
LEFT JOIN `coral-shift-440513-f3.meat_package.payment_transactions_final` pt
  ON o.id = pt.order_id
 AND pt.provider = 'p24'
LEFT JOIN `coral-shift-440513-f3.meat_package.customer_package_deliveries_final` d
  ON o.product_id = d.id
LEFT JOIN `coral-shift-440513-f3.meat_package.customer_package_delivery_products_final` p
  ON d.id = p.customer_package_delivery_id

LIMIT 10;
"""

query_job = client.query(query)
results = query_job.result()

for row in results:
    print(
        # orders_final
        f"order_id: {row.order_id}, "
        f"order_number: {row.order_number}, "
        f"customer_id: {row.customer_id}, "
        f"product_id: {row.product_id}, "
        f"order_price: {row.order_price}, "
        f"product_type: {row.product_type}, "
        f"order_buy_method: {row.order_buy_method}, "
        f"discount_code_id: {row.discount_code_id}, "
        f"order_created_at: {row.order_created_at}, "
        f"order_updated_at: {row.order_updated_at}, "
        f"order_deleted_at: {row.order_deleted_at}, "
        f"order_created_by: {row.order_created_by}, "

        # payment_transactions_final
        f"payment_id: {row.payment_id}, "
        f"payment_order_id: {row.payment_order_id}, "
        f"provider: {row.provider}, "
        f"payment_status: {row.payment_status}, "
        f"customer_status: {row.customer_status}, "
        f"customer_stage: {row.customer_stage}, "
        f"payment_buy_method: {row.payment_buy_method}, "
        f"payment_price: {row.payment_price}, "
        f"payment_created_at: {row.payment_created_at}, "
        f"payment_updated_at: {row.payment_updated_at}, "
        f"payment_deleted_at: {row.payment_deleted_at}, "

        # customer_package_deliveries_final
        f"delivery_id: {row.delivery_id}, "
        f"delivery_number: {row.delivery_number}, "
        f"customer_package_id: {row.customer_package_id}, "
        f"delivery_status: {row.delivery_status}, "
        f"delivery_date: {row.delivery_date}, "
        f"delivery_buy_method: {row.delivery_buy_method}, "
        f"delivery_courier_type: {row.delivery_courier_type}, "
        f"delivery_customer_first_name: {row.delivery_customer_first_name}, "
        f"delivery_created_at: {row.delivery_created_at}, "
        f"delivery_updated_at: {row.delivery_updated_at}, "
        f"delivery_deleted_at: {row.delivery_deleted_at}, "

        # customer_package_delivery_products_final
        f"product_row_id: {row.product_row_id}, "
        f"customer_package_delivery_id: {row.customer_package_delivery_id}, "
        f"delivery_product_id: {row.delivery_product_id}, "
        f"product_type: {row.product_type}, "
        f"quantity: {row.quantity}, "
        f"pack_quantity: {row.pack_quantity}, "
        f"migration_id: {row.migration_id}, "
        f"wms_id: {row.wms_id}, "
        f"product_created_at: {row.product_created_at}, "
        f"product_updated_at: {row.product_updated_at}, "
        f"product_deleted_at: {row.product_deleted_at}"
    )

order_id: 4f9f0aa1-d4d5-4eee-9fc8-7ffbc8550209, order_number: O8958F9A, customer_id: 00000000-0000-0000-0002-7ca2622f5b75, product_id: 38ab8330-3875-483a-8c11-30f8b2eacf10, order_price: 32900, product_type: App\Models\CustomerPackageDelivery, order_buy_method: subscription, discount_code_id: None, order_created_at: 2024-11-06 08:11:39+00:00, order_updated_at: 2024-11-06 08:11:45+00:00, order_deleted_at: None, order_created_by: f9e99116-c179-4fba-8033-2f3b3422cf0d, payment_id: None, payment_order_id: None, provider: None, payment_status: None, customer_status: None, customer_stage: None, payment_buy_method: None, payment_price: None, payment_created_at: None, payment_updated_at: None, payment_deleted_at: None, delivery_id: None, delivery_number: None, customer_package_id: None, delivery_status: None, delivery_date: None, delivery_buy_method: None, delivery_courier_type: None, delivery_customer_first_name: None, delivery_created_at: None, delivery_updated_at: None, delivery_deleted_at: N

In [ ]:
%%sql

SELECT
  o.*,

  pt.*,

  d.*,

  p.*

FROM `coral-shift-440513-f3.meat_package.orders_final` o
LEFT JOIN `coral-shift-440513-f3.meat_package.payment_transactions_final` pt
  ON o.id = pt.order_id
 AND pt.provider = 'p24'
LEFT JOIN `coral-shift-440513-f3.meat_package.customer_package_deliveries_final` d
  ON o.product_id = d.id
LEFT JOIN `coral-shift-440513-f3.meat_package.customer_package_delivery_products_final` p
  ON d.id = p.customer_package_delivery_id

##Zadanie 2:

Ile zamówień Paczek zostało opłaconych w listopadzie 2024 roku?
Ile z nich zostało dostarczonych w listopadzie?
Który produkt znalazł się najczęściej w zamówieniach opłaconych w lutym 2025?


In [ ]:
query = """
SELECT
  COUNT(DISTINCT o.id) AS paid_orders_nov_2024
FROM `coral-shift-440513-f3.meat_package.orders_final` o
JOIN `coral-shift-440513-f3.meat_package.payment_transactions_final` pt
  ON o.id = pt.order_id
WHERE pt.provider = 'p24'
  AND pt.status = 'completed'
  AND EXTRACT(YEAR FROM pt.created_at) = 2024
  AND EXTRACT(MONTH FROM pt.created_at) = 11
  AND o.product_type = 'App\\\\Models\\\\CustomerPackageDelivery';
"""

query_job = client.query(query)
results = query_job.result()

for row in results:
    print(f"Ile zamówień Paczek zostało opłaconych w listopadzie 2024 roku? → {row.paid_orders_nov_2024}")

Ile zamówień Paczek zostało opłaconych w listopadzie 2024 roku? → 1614


In [ ]:
query = """
SELECT
  COUNT(DISTINCT o.id) AS delivered_orders_nov_2024
FROM `coral-shift-440513-f3.meat_package.orders_final` o
JOIN `coral-shift-440513-f3.meat_package.payment_transactions_final` pt
  ON o.id = pt.order_id
JOIN `coral-shift-440513-f3.meat_package.customer_package_deliveries_final` d
  ON o.product_id = d.id
WHERE pt.provider = 'p24'
  AND pt.status = 'completed'
  AND EXTRACT(YEAR FROM pt.created_at) = 2024
  AND EXTRACT(MONTH FROM pt.created_at) = 11
  AND o.product_type = 'App\\\\Models\\\\CustomerPackageDelivery'
  AND (
        LOWER(d.status) = 'delivered'
        OR DATE(d.created_at) BETWEEN '2024-11-01' AND '2024-11-30'
        OR DATE(d.date) BETWEEN '2024-11-01' AND '2024-11-30'
      );
"""

query_job = client.query(query)
results = query_job.result()

for row in results:
    print(f"Ile z nich zostało dostarczonych w listopadzie 2024 roku? → {row.delivered_orders_nov_2024}")

Ile z nich zostało dostarczonych w listopadzie 2024 roku? → 1590


In [ ]:
query = """
SELECT
  p.product_id,
  COUNT(DISTINCT o.id) AS orders_with_product
FROM `coral-shift-440513-f3.meat_package.orders_final` o
JOIN `coral-shift-440513-f3.meat_package.payment_transactions_final` pt
  ON o.id = pt.order_id
JOIN `coral-shift-440513-f3.meat_package.customer_package_deliveries_final` d
  ON o.product_id = d.id
JOIN `coral-shift-440513-f3.meat_package.customer_package_delivery_products_final` p
  ON d.id = p.customer_package_delivery_id
WHERE pt.provider = 'p24'
  AND pt.status = 'completed'
  AND EXTRACT(YEAR FROM pt.created_at) = 2025
  AND EXTRACT(MONTH FROM pt.created_at) = 2
  AND o.product_type = 'App\\\\Models\\\\CustomerPackageDelivery'
GROUP BY p.product_id
ORDER BY orders_with_product DESC
LIMIT 1;
"""

query_job = client.query(query)
results = query_job.result()

for row in results:
    print(f"Który produkt znalazł się najczęściej w zamówieniach opłaconych w lutym 2025? → produkt_id: {row.product_id}, liczba zamówień: {row.orders_with_product}")

Który produkt znalazł się najczęściej w zamówieniach opłaconych w lutym 2025? → produkt_id: ffffffff-ffff-ffff-8087-cb734977c712, liczba zamówień: 911


##Zadanie 3:

Jaki % opłaconych zamówień ma użyty kod zniżkowy w pierwszym zamówieniu?
Jaki % opłaconych zamówień ma użyty kod zniżkowy w trzecim zamówieniu?


In [ ]:
from google.cloud import bigquery
client = bigquery.Client()

query = """
WITH paid_orders AS (
  SELECT o.*
  FROM `coral-shift-440513-f3.meat_package.orders_final` o
  WHERE EXISTS (
    SELECT 1
    FROM `coral-shift-440513-f3.meat_package.payment_transactions_final` pt
    WHERE pt.order_id = o.id
      AND pt.provider = 'p24'
      AND pt.status = 'completed'
  )
),

ranked_orders AS (
  SELECT
    id,
    customer_id,
    discount_code_id,
    created_at,
    ROW_NUMBER() OVER (PARTITION BY customer_id ORDER BY created_at ASC, id ASC) AS order_rank
  FROM paid_orders
  WHERE product_type = 'App\\\\Models\\\\CustomerPackageDelivery'
)

SELECT
  order_rank,
  SAFE_DIVIDE(CAST(COUNTIF(discount_code_id IS NOT NULL) AS FLOAT64), COUNT(*)) * 100 AS pct_with_discount
FROM ranked_orders
GROUP BY order_rank
ORDER BY order_rank;
"""

query_job = client.query(query)
results = query_job.result()

pct_first = None
pct_third = None

for row in results:
    if row.order_rank == 1:
        pct_first = row.pct_with_discount
    elif row.order_rank == 3:
        pct_third = row.pct_with_discount

print(f"Jaki % opłaconych zamówień ma użyty kod zniżkowy w pierwszym zamówieniu? → {pct_first:.2f}%")
print(f"Jaki % opłaconych zamówień ma użyty kod zniżkowy w trzecim zamówieniu? → {pct_third:.2f}%")

Jaki % opłaconych zamówień ma użyty kod zniżkowy w pierwszym zamówieniu? → 45.24%
Jaki % opłaconych zamówień ma użyty kod zniżkowy w trzecim zamówieniu? → 0.00%


##Zadanie 4:

Zaproponuj KPI, przeprowadź analizę i zaprezentuj wyniki przedstawiające:
Powracalność klientów – np. analiza kohortowa (ile % klientów wraca po pierwszym zakupie, w jakim czasie)
Wartość klienta (LTV) – szacunkowa wartość klienta w czasie
Zaproponuj do powyższych zagadnień wymiary biznesowe – podziały, które Twoim zdaniem są najbardziej istotne z punktu widzenia analizy / monitorowania

###Analiza kohortowa (powracalność miesięczna)

In [ ]:
from google.cloud import bigquery

client = bigquery.Client()

query = """
WITH paid_orders AS (
  SELECT
    o.id AS order_id,
    o.customer_id,
    DATE(o.created_at) AS order_date,
    DATE_TRUNC(DATE(o.created_at), MONTH) AS order_month
  FROM `coral-shift-440513-f3.meat_package.orders_final` o
  WHERE o.product_type = 'App\\\\Models\\\\CustomerPackageDelivery'
    AND EXISTS (
      SELECT 1 FROM `coral-shift-440513-f3.meat_package.payment_transactions_final` pt
      WHERE pt.order_id = o.id
        AND pt.provider = 'p24'
        AND pt.status = 'completed'
    )
),

first_orders AS (
  SELECT
    customer_id,
    MIN(order_date) AS first_order_date,
    DATE_TRUNC(MIN(order_date), MONTH) AS cohort_month
  FROM paid_orders
  GROUP BY customer_id
),

orders_with_cohort AS (
  SELECT
    po.*,
    fo.first_order_date,
    fo.cohort_month,
    DATE_DIFF(DATE_TRUNC(po.order_date, MONTH), fo.cohort_month, MONTH) AS months_since_cohort
  FROM paid_orders po
  JOIN first_orders fo
    ON po.customer_id = fo.customer_id
)

SELECT
  cohort_month,
  months_since_cohort,
  COUNT(DISTINCT customer_id) AS customers_in_period,
  (SELECT COUNT(1) FROM first_orders fo WHERE fo.cohort_month = owc.cohort_month) AS cohort_size,
  SAFE_DIVIDE(COUNT(DISTINCT customer_id),
              (SELECT COUNT(1) FROM first_orders fo WHERE fo.cohort_month = owc.cohort_month)) AS retention_rate
FROM orders_with_cohort owc
GROUP BY cohort_month, months_since_cohort
ORDER BY cohort_month, months_since_cohort
"""

query_job = client.query(query)
results = query_job.result()

for row in results:
    print(
        f"Kohorta: {row.cohort_month}, "
        f"Miesiące od pierwszego zakupu: {row.months_since_cohort}, "
        f"Liczba klientów: {row.customers_in_period}, "
        f"Rozmiar kohorty: {row.cohort_size}, "
        f"Retencja: {round(row.retention_rate * 100, 2)}%"
    )

Kohorta: 2023-08-01, Miesiące od pierwszego zakupu: 0, Liczba klientów: 468, Rozmiar kohorty: 468, Retencja: 100.0%
Kohorta: 2023-08-01, Miesiące od pierwszego zakupu: 1, Liczba klientów: 158, Rozmiar kohorty: 468, Retencja: 33.76%
Kohorta: 2023-08-01, Miesiące od pierwszego zakupu: 2, Liczba klientów: 182, Rozmiar kohorty: 468, Retencja: 38.89%
Kohorta: 2023-08-01, Miesiące od pierwszego zakupu: 3, Liczba klientów: 173, Rozmiar kohorty: 468, Retencja: 36.97%
Kohorta: 2023-08-01, Miesiące od pierwszego zakupu: 4, Liczba klientów: 142, Rozmiar kohorty: 468, Retencja: 30.34%
Kohorta: 2023-08-01, Miesiące od pierwszego zakupu: 5, Liczba klientów: 172, Rozmiar kohorty: 468, Retencja: 36.75%
Kohorta: 2023-08-01, Miesiące od pierwszego zakupu: 6, Liczba klientów: 151, Rozmiar kohorty: 468, Retencja: 32.26%
Kohorta: 2023-08-01, Miesiące od pierwszego zakupu: 7, Liczba klientów: 155, Rozmiar kohorty: 468, Retencja: 33.12%
Kohorta: 2023-08-01, Miesiące od pierwszego zakupu: 8, Liczba klientów: 

Analiza kohortowa pokazuje, jak klienci pozyskani w danym okresie wracają z kolejnymi zakupami w kolejnych miesiącach. W danych widać, że kohorty z drugiej połowy 2023 roku utrzymują relatywnie wyższą retencję (ok. 20–30% po kilku miesiącach), podczas gdy kohorty z końca 2024 i początku 2025 roku szybko tracą klientów (często poniżej 15% już po 2–3 miesiącach). Oznacza to, że jakość pozyskanych klientów w nowszych kohortach spadła lub zmieniła się oferta bądź kanały akwizycji. Biznesowo sugeruje to potrzebę optymalizacji kampanii marketingowych i weryfikacji źródeł ruchu, które mogą przyciągać mniej lojalnych klientów. Warto też przyjrzeć się strategii utrzymania (np. komunikacji, promocjom dla powracających), aby spowolnić spadek retencji. Stały trend spadkowy w retencji w ostatnich miesiącach może wskazywać na pogorszenie doświadczenia klienta, problem z produktem lub brak mechanizmów lojalnościowych.

###Wartość klienta (LTV)

In [ ]:
from google.cloud import bigquery
client = bigquery.Client()

query = """
WITH paid_orders AS (
  SELECT
    o.customer_id,
    DATE_TRUNC(DATE(o.created_at), MONTH) AS cohort_month,
    SUM(o.price) / 100.0 AS order_value
  FROM `coral-shift-440513-f3.meat_package.orders_final` o
  JOIN `coral-shift-440513-f3.meat_package.payment_transactions_final` pt
    ON o.id = pt.order_id
   AND pt.provider = 'p24'
   AND pt.status = 'completed'
  WHERE o.product_type = 'App\\\\Models\\\\CustomerPackageDelivery'
  GROUP BY o.customer_id, cohort_month, o.id
),

customer_revenue AS (
  SELECT
    customer_id,
    cohort_month,
    SUM(order_value) AS customer_cumulative_revenue
  FROM paid_orders
  GROUP BY customer_id, cohort_month
),

cohort_ltv AS (
  SELECT
    cohort_month,
    AVG(customer_cumulative_revenue) AS avg_ltv
  FROM customer_revenue
  GROUP BY cohort_month
)

SELECT *
FROM cohort_ltv
ORDER BY cohort_month
"""

query_job = client.query(query)
results = query_job.result()

for row in results:
    print(
        f"Kohorta: {row.cohort_month}, "
        f"Średnie LTV: {round(row.avg_ltv, 2)} PLN"
    )

Kohorta: 2023-08-01, Średnie LTV: 453.75 PLN
Kohorta: 2023-09-01, Średnie LTV: 434.43 PLN
Kohorta: 2023-10-01, Średnie LTV: 429.4 PLN
Kohorta: 2023-11-01, Średnie LTV: 427.81 PLN
Kohorta: 2023-12-01, Średnie LTV: 436.74 PLN
Kohorta: 2024-01-01, Średnie LTV: 440.45 PLN
Kohorta: 2024-02-01, Średnie LTV: 429.94 PLN
Kohorta: 2024-03-01, Średnie LTV: 447.07 PLN
Kohorta: 2024-04-01, Średnie LTV: 450.04 PLN
Kohorta: 2024-05-01, Średnie LTV: 456.84 PLN
Kohorta: 2024-06-01, Średnie LTV: 446.41 PLN
Kohorta: 2024-07-01, Średnie LTV: 454.7 PLN
Kohorta: 2024-08-01, Średnie LTV: 457.54 PLN
Kohorta: 2024-09-01, Średnie LTV: 464.4 PLN
Kohorta: 2024-10-01, Średnie LTV: 465.72 PLN
Kohorta: 2024-11-01, Średnie LTV: 481.81 PLN
Kohorta: 2024-12-01, Średnie LTV: 500.15 PLN
Kohorta: 2025-01-01, Średnie LTV: 499.04 PLN
Kohorta: 2025-02-01, Średnie LTV: 483.59 PLN
Kohorta: 2025-03-01, Średnie LTV: 516.9 PLN
Kohorta: 2025-04-01, Średnie LTV: 515.45 PLN
Kohorta: 2025-05-01, Średnie LTV: 516.34 PLN
Kohorta: 2025-

Na podstawie danych widać, że średnie LTV klientów stopniowo rośnie – od około 430–450 PLN w drugiej połowie 2023 roku do poziomów przekraczających 500 PLN w 2025 roku. To sugeruje, że firma z czasem zaczęła pozyskiwać bardziej wartościowych klientów lub skuteczniej monetyzować już istniejących. Widać też, że końcówka 2024 roku i początek 2025 to wyraźny skok w średnim LTV, co może wskazywać na zmianę w ofercie, strategii cenowej lub kanałach akwizycji. Warto zauważyć, że najnowsze kohorty (lipiec–sierpień 2025) pokazują lekkie obniżenie LTV, co może wynikać ze spadku jakości pozyskiwanego ruchu. Trend ogólny jest jednak pozytywny i sugeruje coraz lepsze dopasowanie klientów do produktu.

###Dalsze cięcie danych

**Retencja (Cohort Retention)**

- Wymiary analizy:
    - Kanał pozyskania klienta (acquisition channel)
	    - Google Ads, Meta Ads, organic search, polecenia, partnerzy. Pozwala ocenić jakość pozyskiwanych użytkowników.
	- Segment produktu / pakiet
	    - np. „5 porcji”, „10 porcji” albo rodzaj mięsa. Zobaczysz, które pakiety najlepiej przyciągają lojalnych klientów.
	- Typ klienta / profil demograficzny
	    - B2B vs B2C, gospodarstwa 1-osobowe vs rodziny, płeć. Różnice w powrotach mogą być ogromne.
	- Region / lokalizacja
	    - Województwo, miasto, region dostawy. W logistyce i subskrypcjach lokalizacja często ma krytyczne znaczenie.
	- Kanał sprzedaży
	    - Mobile vs desktop, aplikacja vs strona www. Klienci aplikacyjni często mają wyższy retention.
	- Forma płatności
	    - Płatność cykliczna (subskrypcja, autopay) vs manualna (jednorazowa płatność). Autopay podbija retencję.

**LTV (Lifetime Value)**

- Wymiary analizy:
	- Cohort Month (już masz)
	    - LTV kohortowy pozwala patrzeć na trend jakości klientów w czasie.
	- Kanał pozyskania
	    - Kluczowe: mierzyć CAC (Customer Acquisition Cost) vs LTV (Customer Lifetime Value), żeby policzyć ROI kanałów. W e-commerce często mówi się, że LTV powinien być co najmniej 3× większy niż CAC, żeby kanał był opłacalny (tzw. reguła 3:1).
	- Typ produktu / pakiet
	    - Które pakiety generują najwyższe LTV? Może droższe pakiety mają mniejszą retencję, ale wyższy średni przychód?
	- Segment lojalnościowy
	    - Nowi klienci vs powracający (np. 2+ zamówienia).
	- Typ dostawy / logistyka / czas dostawy
	    - Czasem wygoda = większe LTV.
	- Pozyskanie nowych danych
	    - NPS
		- Ankiety np. katering dietetyczny zaprasza mnie do ankiet średnio co miesiąc. Dzięki temu mogą ustalić szerszy kontekst kim jest klient, jakie posiada zwyczaje nie tylko żywieniowe.



Najważniejsze biznesowo wymiary do monitorowania to kanał pozyskania, pakiet/produkt, promocja, typ klienta i lokalizacja. To one najczęściej pokazują największe różnice w retencji i LTV.

#Część 2

**Zaproponuj podejście analityczne, które pomoże zrozumieć, co powoduje brak korelacji między wydatkami marketingowymi/reklamowymi a wynikami sprzedaży.**

- Głównym problemem jest prawdopodobnie brak pełnego obrazu drogi klienta przez wszystkie punkty kontaktu z firmą. Tradycyjne mierzenie efektywności (ROAS, ERS) nie uwzględnia wpływu jakości obsługi klienta na końcowe rezultaty sprzedażowe.
Proponuję podejście oparte na mapowaniu pełnej ścieżki klienta - od pierwszego kontaktu z reklamą, przez interakcje z contentem, obsługą klienta, aż po finalizację zakupu i retention. To pozwoli zidentyfikować, gdzie tracą się potencjalni klienci i dlaczego rosnące wydatki nie przekładają się na proporcjonalny wzrost sprzedaży.


**Określ, jakie dane chciałbyś/-abyś zebrać, z jakich systemów i jak je powiązać, by uzyskać pełny obraz sytuacji.**



- Marketing i reklama:

    - Google Ads - wydatki, kliki, konwersje z podziałem na typy kampanii
    - Meta Ads - reach, frequency, koszt pozyskania ruchu
    - Email marketing - otwarcia, kliki, konwersje z mailingu
    - Dane o kampaniach brandowych vs. performance

- Obsługa klienta i telesprzedaż:

    - Liczba zgłoszeń serwisowych na klienta
    - Czas odpowiedzi i rozwiązywania problemów
    - Oceny satysfakcji klientów NPS
    - Skuteczność telesprzedaży (konwersja połączeń)

- Sprzedaż i zachowania:

    - Wartość zamówień z podziałem na źródło pozyskania
    - Dane o porzuconych koszykach
    - Frequency zakupów (czy klienci wracają?)
    - Analiza ruchu na stronie (bounce rate, czas sesji)

- Dane operacyjne:

    - Koszty personelu każdego działu
    - Koszty narzędzi marketingowych
    - Timing kampanii i działań każdego działu

1. Wdróż tabelę mapping (user_pseudo_id / Custom First-Party ID ↔ user_id ↔ crm_id) w BigQuery i backfilluj historyczne eventy. ￼
2.  Uruchom server-side tagging + persistent first-party id, żeby poprawić spójność identyfikatorów.
3. Wdróż narzędzie trzecie do monitoringu za pomocą ML podejrzanego ruchu (boty). Zazwyczaj pada na sieci afiliacyjne.

**Wskaż, jakie wskaźniki (KPI) i metryki są kluczowe w analizie efektywności poszczególnych kanałów i ich wpływu na sprzedaż.**

**Zamiast podstawowych metryk, focus na:**
 - True Customer Acquisition Cost (True CAC)

    - Uwzględniający nie tylko media spend, ale koszty personelu, narzędzi, overhead

- Marketing Efficiency Ratio (MER)

    - Całkowity przychód / całkowite wydatki marketingowe

- Service Impact Score

    - Korelacja między jakością obsługi a poziomem retention klientów
    - Wpływ czasu odpowiedzi na konwersję
    - Słaba obsługa = wyższy churn (odpływ klientów), co psuje LTV

- Cross-Channel Attribution Score

    - Rzeczywisty udział każdego kanału w generowaniu sprzedaży.
    - Uwzględniający efekt synergii między kanałami.
    - Testy Conversion lift.

- Customer Journey Completion Rate
    - Jaki % klientów przechodzi przez wszystkie etapy ścieżki bez "wypadania"
    - Pokazuje, w którym miejscu najwięcej klientów odpada. To daje konkretne wskazówki do optymalizacji (UX, formularze, płatności, logistyka).

**Zaproponuj hipotetyczne wnioski lub zależności, które chciałbyś/-abyś zweryfikować (np. wpływ jakości obsługi klienta na sprzedaż, zależność między typem kampanii a konwersją itd.).**

- H1: Kanibalizacja kanałów
Kampanie płatne "kradną" ruch, który i tak by przyszedł organicznie. Klienci szukający marki trafiają na płatne reklamy zamiast wchodzić bezpośrednio.
    - Jak sprawdzić: Porównaj wzrost brandowych kliknięć płatnych z trendem wejść organicznych/bezpośrednich.
- H2: Obsługa niweluje efekty kampanii
Słaba obsługa klienta (długi czas odpowiedzi, niska satysfakcja) powoduje, że klienci pozyskani przez drogie kampanie rezygnują z zakupów lub nie wracają.
    - Jak sprawdzić: Segmentacja retention wg NPS lub czasu odpowiedzi supportu.
- H3: Błędna atrybucja
System przypisuje konwersje ostatniemu kliknięciu, ignorując wpływ kampanii brandowych i mailingów, które "przygotowują" klienta do zakupu.
    - Jak sprawdzić: Sprawedź (data-driven w GA4, eksperymenty z wyłączeniem kanału).
- H4: Brak synchronizacji działań
Działy działają w silosach - np. telesprzedaż dzwoni do klientów, którzy nie otrzymali jeszcze follow-up emaili, lub kampanie reklamowe nie są zsynchronizowane z dostępnością produktów.
    - Jak sprawdzić: Analiza kampanii vs. stock levels + korelacja z CR i churnem.
- H5: Jakość ruchu vs. ilość
Focus na zwiększanie ruchu kosztem jego jakości - trafiają klienci o niskiej skłonności do zakupu.
    - Jak sprawdzić: Analiza konwersji i retention per źródło ruchu. Porównaj np. Paid Social vs. Paid Search vs. Organic.

**Na koniec, zaproponuj konkretne działania lub usprawnienia, które Twoim zdaniem warto byłoby przetestować w organizacji, aby poprawić efektywność działań sprzedażowo-marketingowych.**

**Działania opercyjne**

1. Unified Customer ID - Implementacja systemu łączącego wszystkie interakcje klienta - od reklamy, przez email, obsługę, do zakupu. To podstawa do prawdziwej analizy efektywności.
2. Service Quality Alert System - Automatyczne alerty gdy jakość obsługi spada - bo może to niwelować efekty drogich kampanii pozyskania.
3. Weekly Cross-Department Sync - Cotygodniowe spotkania wszystkich działów z dashboardem pokazującym wspólne KPI - eliminacja pracy w silosach.

**Działania średnioterminowe:**

4. Advanced Attribution Model - Przejście z "last-click" na model uwzględniający wszystkie touchpointy + wpływ obsługi klienta na finalną konwersję.
5. Intelligent Campaign Sequencing - System automatycznie synchronizujący działania - np. follow-up email 24h po kliknięciu w reklamę, telemarketing 3 dni po porzuceniu koszyka.
6. Incrementality Testing Framework - Regularne testy wyłączania poszczególnych kanałów w wybranych regionach/segmentach żeby zmierzyć rzeczywisty wpływ na sprzedaż.


**Kluczowy wniosek:**

Problem prawdopodobnie nie leży w samych kampaniach, ale w braku holistycznego podejścia do customer experience.